##### <b>Install Library</b>

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import time

import warnings
warnings.filterwarnings("ignore")

##### <b>SET50 Index Futures</b>

In [2]:
data = pd.read_csv('SET50_Index_Futures.csv')
data = data[(data['Symbol'] != 'S50M06') & (data['Symbol'] != 'S50H07') & (data['Symbol'] != 'S50M07') & (data['Symbol'] != 'S50U07')].reset_index(drop=True)
data = data.pivot(index='Date', columns='Symbol', values='SP').reset_index().drop_duplicates(subset='Date', keep='last')
data['Date'] = pd.to_datetime(data['Date'])
data = data[data['Date'] >= '2007-10-29'].reset_index(drop=True)
data

Symbol,Date,S50H08,S50H09,S50H10,S50H11,S50H12,S50H13,S50H14,S50H15,S50H16,...,S50Z14,S50Z15,S50Z16,S50Z17,S50Z18,S50Z19,S50Z20,S50Z21,S50Z22,S50Z23
0,2007-10-29,693.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007-10-30,681.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2007-10-31,688.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007-11-01,677.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2007-11-02,673.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3923,2023-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,853.7
3924,2023-12-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,856.3
3925,2023-12-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,856.6
3926,2023-12-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,849.4


##### <b>Manipulate SET50 Index Futures</b>

In [3]:
def round_to_nearest_25(value):
    return 25 * round(value / 25)

def round_to_nearest_50(value):
    return 50 * round(value / 50)

def vlookup_call(row, file_prefix):
    strike_price = row['Strike Price']
    column_name = f'{file_prefix}C{strike_price}'
    atm_value = row[column_name]

    return atm_value

def vlookup_put(row, file_prefix):
    strike_price = row['Strike Price']
    column_name = f'{file_prefix}P{strike_price}'
    atm_value = row[column_name]

    return atm_value

column_names_25 = ['S50H14', 'S50H15', 'S50H16', 'S50H17', 'S50H18', 'S50H19', 'S50H20', 'S50H21', 'S50H22', 'S50H23', 
                   'S50M14', 'S50M15', 'S50M16', 'S50M17', 'S50M18', 'S50M19', 'S50M20', 'S50M21', 'S50M22', 'S50M23', 
                   'S50U13', 'S50U14', 'S50U15', 'S50U16', 'S50U17', 'S50U18', 'S50U19', 'S50U20', 'S50U21', 'S50U22', 'S50U23', 
                   'S50Z13', 'S50Z14', 'S50Z15', 'S50Z16', 'S50Z17', 'S50Z18', 'S50Z19', 'S50Z20', 'S50Z21', 'S50Z22', 'S50Z23']
    
column_names_50 = ['S50H08', 'S50H09', 'S50H10', 'S50H11', 'S50H12', 'S50H13',
                   'S50M08', 'S50M09', 'S50M10', 'S50M11', 'S50M12', 'S50M13',
                   'S50U06', 'S50U08', 'S50U09', 'S50U10', 'S50U11', 'S50U12',
                   'S50Z06', 'S50Z07', 'S50Z08', 'S50Z09', 'S50Z10', 'S50Z11', 'S50Z12']

# Dictionary to store DataFrames
dfs_25 = {}
dfs_50 = {}

# Create DataFrames for 'column_names_25'
for column in column_names_25:
    selected_columns = ['Date', column]
    new_df_25 = data[selected_columns].dropna().reset_index(drop=True)
    
    # Calculate Strike_Price column with the round_to_nearest_25 function
    new_df_25['Strike Price'] = new_df_25[column].apply(round_to_nearest_25)
    
    # Store the DataFrame in the dictionary
    dfs_25[column] = new_df_25

# Create DataFrames for 'column_names_50'
for column in column_names_50:
    selected_columns = ['Date', column]
    new_df_50 = data[selected_columns].dropna().reset_index(drop=True)
    
    # Calculate Strike_Price column with the round_to_nearest_50 function
    new_df_50['Strike Price'] = new_df_50[column].apply(round_to_nearest_50)
    
    # Store the DataFrame in the dictionary
    dfs_50[column] = new_df_50

# Interval 25
S50H14 = dfs_25['S50H14']; S50H15 = dfs_25['S50H15']; S50H16 = dfs_25['S50H16']; S50H17 = dfs_25['S50H17']
S50H18 = dfs_25['S50H18']; S50H19 = dfs_25['S50H19']; S50H20 = dfs_25['S50H20']; S50H21 = dfs_25['S50H21']
S50H22 = dfs_25['S50H22']; S50H23 = dfs_25['S50H23']

S50M14 = dfs_25['S50M14']; S50M15 = dfs_25['S50M15']; S50M16 = dfs_25['S50M16']; S50M17 = dfs_25['S50M17']
S50M18 = dfs_25['S50M18']; S50M19 = dfs_25['S50M19']; S50M20 = dfs_25['S50M20']; S50M21 = dfs_25['S50M21']
S50M22 = dfs_25['S50M22']; S50M23 = dfs_25['S50M23']

S50U13 = dfs_25['S50U13']; S50U14 = dfs_25['S50U14']; S50U15 = dfs_25['S50U15']; S50U16 = dfs_25['S50U16']
S50U17 = dfs_25['S50U17']; S50U18 = dfs_25['S50U18']; S50U19 = dfs_25['S50U19']; S50U20 = dfs_25['S50U20']
S50U21 = dfs_25['S50U21']; S50U22 = dfs_25['S50U22']; S50U23 = dfs_25['S50U23']

S50Z13 = dfs_25['S50Z13']; S50Z14 = dfs_25['S50Z14']; S50Z15 = dfs_25['S50Z15']; S50Z16 = dfs_25['S50Z16']
S50Z17 = dfs_25['S50Z17']; S50Z18 = dfs_25['S50Z18']; S50Z19 = dfs_25['S50Z19']; S50Z20 = dfs_25['S50Z20']
S50Z21 = dfs_25['S50Z21']; S50Z22 = dfs_25['S50Z22']; S50Z23 = dfs_25['S50Z23']

# Interval 50
S50H08 = dfs_50['S50H08']; S50H09 = dfs_50['S50H09']; S50H10 = dfs_50['S50H10']; S50H11 = dfs_50['S50H11']
S50H12 = dfs_50['S50H12']; S50H13 = dfs_50['S50H13']

S50M08 = dfs_50['S50M08']; S50M09 = dfs_50['S50M09']; S50M10 = dfs_50['S50M10']; S50M11 = dfs_50['S50M11']
S50M12 = dfs_50['S50M12']; S50M13 = dfs_50['S50M13']

S50U08 = dfs_50['S50U08']; S50U09 = dfs_50['S50U09']; S50U10 = dfs_50['S50U10']; S50U11 = dfs_50['S50U11']
S50U12 = dfs_50['S50U12']

S50Z07 = dfs_50['S50Z07']; S50Z08 = dfs_50['S50Z08']; S50Z09 = dfs_50['S50Z09']; S50Z10 = dfs_50['S50Z10']
S50Z11 = dfs_50['S50Z11']; S50Z12 = dfs_50['S50Z12']

##### <b>Combined SET50 Index Futures and SET50 Index Options</b>

###### <b>Create Functions</b>

In [4]:
def process_options_data(futures_symbol):
    Call_Options_data = pd.read_csv(f'{futures_symbol}_Call_options_data.csv')
    Call_Options = Call_Options_data[['Date', 'Symbol', 'SP']]
    Call_Options = Call_Options.sort_values(by='Date').reset_index(drop=True)

    Put_Options_data = pd.read_csv(f'{futures_symbol}_Put_options_data.csv')
    Put_Options = Put_Options_data[['Date', 'Symbol', 'SP']]
    Put_Options = Put_Options.sort_values(by='Date').reset_index(drop=True)

    # Select options first day trade
    first_day = Call_Options['Date'].min()
    Futures['Date'] = pd.to_datetime(Futures['Date'])
    mask = Futures['Date'] >= pd.to_datetime(first_day)
    Futures_subset = Futures[mask].reset_index(drop=True)
    Futures_subset['Date'] = pd.to_datetime(Futures_subset['Date'])
    Futures_subset = Futures_subset.sort_values(by='Date').reset_index(drop=True)

    call_options_unique_symbols = Call_Options['Symbol'].unique()
    symbol_dataframes = {}

    for symbol in call_options_unique_symbols:
        symbol_df = pd.DataFrame({
            'Date': Call_Options[Call_Options['Symbol'] == symbol]['Date'],
            symbol: Call_Options[Call_Options['Symbol'] == symbol]['SP']
        })
        symbol_dataframes[symbol] = symbol_df

    merged_df = symbol_dataframes[call_options_unique_symbols[0]]
    for symbol in call_options_unique_symbols[1:]:
        merged_df = pd.merge(merged_df, symbol_dataframes[symbol], on='Date', how='outer')

    Call_Options_Clean = merged_df.drop_duplicates(subset=['Date']).sort_values(by='Date').reset_index(drop=True).fillna(0)
    Call_Options_Clean['Date'] = pd.to_datetime(Call_Options_Clean['Date'])

    put_options_unique_symbols = Put_Options['Symbol'].unique()
    symbol_dataframes = {}

    for symbol in put_options_unique_symbols:
        symbol_df = pd.DataFrame({
            'Date': Put_Options[Put_Options['Symbol'] == symbol]['Date'],
            symbol: Put_Options[Put_Options['Symbol'] == symbol]['SP']
        })
        symbol_dataframes[symbol] = symbol_df

    merged_df = symbol_dataframes[put_options_unique_symbols[0]]
    for symbol in put_options_unique_symbols[1:]:
        merged_df = pd.merge(merged_df, symbol_dataframes[symbol], on='Date', how='outer')

    Put_Options_Clean = merged_df.drop_duplicates(subset=['Date']).sort_values(by='Date').reset_index(drop=True).fillna(0)
    Put_Options_Clean['Date'] = pd.to_datetime(Put_Options_Clean['Date'])

    Data = Futures_subset.merge(Call_Options_Clean, on='Date', how='outer').fillna(0)
    Data = Data.merge(Put_Options_Clean, on='Date', how='outer').fillna(0)
    Data['Date'] = pd.to_datetime(Data['Date'])
    Data = Data.sort_values(by='Date').reset_index(drop=True)
    Data['Strike Price'] = Data['Strike Price'].astype(int)
    Data['Call ATM'] = Data.apply(vlookup_call, axis=1, args=(futures_symbol,))
    Data['Put ATM'] = Data.apply(vlookup_put, axis=1, args=(futures_symbol,))
    Data = Data[['Date', 'Strike Price', 'Call ATM', 'Put ATM']]
    Data['Sum Premium'] = (Data['Call ATM'] + Data['Put ATM'])
    Data['Normalized Premium'] = Data['Strike Price'] / Data['Sum Premium']

    return Data

###### <b>S50U13 to S50Z23</b>

In [5]:
# 2013
Futures = S50U13
Futures_symbol = 'S50U13'
S50U13 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50Z13
Futures_symbol = 'S50Z13'
S50Z13 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

# Options_2013 = pd.concat([S50H13, S50M13, S50U13, S50Z13], axis=1)
Options_2013 = pd.concat([S50U13, S50Z13], axis=1)
Options_2013

,S50U13,S50Z13
0,11.668611,7.090909
1,11.822376,7.111597
2,11.877173,7.132407
3,11.904762,8.051197
4,11.560694,7.105460
...,...,...
118,36.931818,NaN
119,35.454545,NaN
120,48.994975,NaN
121,65.000000,NaN


In [6]:
# 2014
Futures = S50H14
Futures_symbol = 'S50H14'
S50H14 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50M14
Futures_symbol = 'S50M14'
S50M14 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50U14
Futures_symbol = 'S50U14'
S50U14 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50Z14
Futures_symbol = 'S50Z14'
S50Z14 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Options_2014 = pd.concat([S50H14, S50M14, S50U14, S50Z14], axis=1)
Options_2014

,S50H14,S50M14,S50U14,S50Z14
0,6.709040,9.086189,13.309353,14.513788
1,6.732814,9.898190,11.131167,16.420361
2,6.629449,inf,11.038186,14.836795
3,6.780871,8.667389,11.533666,17.699115
4,6.794425,8.957655,11.664565,18.382353
...,...,...,...,...
120,51.388889,86.956522,94.298246,50.995025
121,75.203252,0.499156,0.498812,47.235023
122,71.705426,NaN,NaN,57.803468
123,0.495108,NaN,NaN,53.763441


In [7]:
# 2015
Futures = S50H15
Futures_symbol = 'S50H15'
S50H15 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50M15
Futures_symbol = 'S50M15'
S50M15 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50U15
Futures_symbol = 'S50U15'
S50U15 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50Z15
Futures_symbol = 'S50Z15'
S50Z15 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Options_2015 = pd.concat([S50H15, S50M15, S50U15, S50Z15], axis=1)
Options_2015

,S50H15,S50M15,S50U15,S50Z15
0,19.195612,9.420290,14.338235,14.508929
1,21.694215,9.779338,13.017356,13.429752
2,22.483940,9.979529,14.409222,13.908702
3,18.388792,10.284810,14.025245,13.636364
4,21.560575,9.842520,14.880952,12.907609
...,...,...,...,...
121,59.880240,NaN,NaN,37.844037
122,80.000000,NaN,NaN,45.329670
123,0.500947,NaN,NaN,50.000000
124,NaN,NaN,NaN,61.111111


In [8]:
# 2016
Futures = S50H16
Futures_symbol = 'S50H16'
S50H16 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50M16
Futures_symbol = 'S50M16'
S50M16 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50U16
Futures_symbol = 'S50U16'
S50U16 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50Z16
Futures_symbol = 'S50Z16'
S50Z16 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Options_2016 = pd.concat([S50H16, S50M16, S50U16, S50Z16], axis=1)
Options_2016

,S50H16,S50M16,S50U16,S50Z16
0,9.626274,9.651308,10.489510,10.856454
1,9.423503,8.123690,11.111111,11.002445
2,10.403917,8.047767,9.449244,12.162162
3,8.517034,8.581236,11.393229,12.534819
4,8.966245,9.270705,9.159483,10.663507
...,...,...,...,...
121,46.153846,NaN,NaN,106.741573
122,57.324841,NaN,NaN,115.853659
123,62.500000,NaN,NaN,186.274510
124,86.538462,NaN,NaN,0.505716


In [9]:
# 2017
Futures = S50H17
Futures_symbol = 'S50H17'
S50H17 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50M17
Futures_symbol = 'S50M17'
S50M17 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50U17
Futures_symbol = 'S50U17'
S50U17 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50Z17
Futures_symbol = 'S50Z17'
S50Z17 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Options_2017 = pd.concat([S50H17, S50M17, S50U17, S50Z17], axis=1)
Options_2017

,S50H17,S50M17,S50U17,S50Z17
0,10.857143,17.924528,24.570025,37.037037
1,10.686164,13.456091,23.955774,41.322314
2,10.063559,14.772727,22.465438,41.666667
3,10.957324,14.401773,23.752969,42.016807
4,10.770975,15.905383,23.696682,41.841004
...,...,...,...,...
120,109.890110,188.679245,109.693878,80.357143
121,106.382979,232.558140,143.333333,85.227273
122,144.927536,0.502462,0.494220,0.499836
123,169.491525,NaN,NaN,NaN


In [10]:
# 2018
Futures = S50H18
Futures_symbol = 'S50H18'
S50H18 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50M18
Futures_symbol = 'S50M18'
S50M18 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50U18
Futures_symbol = 'S50U18'
S50U18 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50Z18
Futures_symbol = 'S50Z18'
S50Z18 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Options_2018 = pd.concat([S50H18, S50M18, S50U18, S50Z18], axis=1)
Options_2018

,S50H18,S50M18,S50U18,S50Z18
0,20.873786,19.067797,12.771739,10.703364
1,20.593870,19.230769,11.758691,11.413043
2,20.168856,17.371601,14.650873,11.602210
3,18.251273,16.714083,17.503805,11.659436
4,18.598616,17.530488,10.954236,12.024609
...,...,...,...,...
119,76.433121,53.571429,68.452381,46.536797
120,76.433121,58.108108,75.657895,55.128205
121,123.684211,65.625000,92.741935,55.405405
122,0.496923,0.494136,0.498716,62.500000


In [11]:
# 2019
Futures = S50H19
Futures_symbol = 'S50H19'
S50H19 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50M19
Futures_symbol = 'S50M19'
S50M19 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50U19
Futures_symbol = 'S50U19'
S50U19 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50Z19
Futures_symbol = 'S50Z19'
S50Z19 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Options_2019 = pd.concat([S50H19, S50M19, S50U19, S50Z19], axis=1)
Options_2019

,S50H19,S50M19,S50U19,S50Z19
0,12.849162,11.647727,17.063492,18.668831
1,13.609467,11.602210,16.955836,19.524618
2,14.197531,10.881104,15.762463,18.883415
3,14.232673,12.424242,17.916667,18.883415
4,14.593909,12.237762,19.366197,19.458545
...,...,...,...,...
120,64.371257,0.498548,88.709677,80.223881
121,59.782609,NaN,0.501395,67.610063
122,72.147651,NaN,NaN,93.478261
123,0.505129,NaN,NaN,122.159091


In [12]:
# 2020
Futures = S50H20
Futures_symbol = 'S50H20'
S50H20 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50M20
Futures_symbol = 'S50M20'
S50M20 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50U20
Futures_symbol = 'S50U20'
S50U20 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50Z20
Futures_symbol = 'S50Z20'
S50Z20 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Options_2020 = pd.concat([S50H20, S50M20, S50U20, S50Z20], axis=1)
Options_2020

,S50H20,S50M20,S50U20,S50Z20
0,14.827586,17.200000,2.429625,7.371525
1,16.337386,17.594108,2.423128,7.582322
2,16.412214,17.338710,2.527013,6.684492
3,15.949555,17.310789,2.561475,7.685739
4,16.312595,15.885023,2.662407,7.487521
...,...,...,...,...
120,13.861386,35.282258,0.503297,0.501306
121,19.078947,41.469194,NaN,NaN
122,21.067416,52.710843,NaN,NaN
123,24.350649,0.499800,NaN,NaN


In [13]:
# 2021
Futures = S50H21
Futures_symbol = 'S50H21'
S50H21 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50M21
Futures_symbol = 'S50M21'
S50M21 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50U21
Futures_symbol = 'S50U21'
S50U21 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50Z21
Futures_symbol = 'S50Z21'
S50Z21 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Options_2021 = pd.concat([S50H21, S50M21, S50U21, S50Z21], axis=1)
Options_2021

,S50H21,S50M21,S50U21,S50Z21
0,10.610080,6.138023,11.271676,15.297907
1,9.910486,6.428571,12.148338,15.625000
2,10.104302,7.335448,12.500000,15.372168
3,10.117493,6.761566,13.087248,14.660494
4,11.008523,6.101583,13.031550,15.079365
...,...,...,...,...
119,55.714286,NaN,0.502437,68.661972
120,87.053571,NaN,NaN,103.723404
121,0.500698,NaN,NaN,130.000000
122,NaN,NaN,NaN,103.723404


In [14]:
# 2022
Futures = S50H22
Futures_symbol = 'S50H22'
S50H22 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50M22
Futures_symbol = 'S50M22'
S50M22 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50U22
Futures_symbol = 'S50U22'
S50U22 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50Z22
Futures_symbol = 'S50Z22'
S50Z22 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Options_2022 = pd.concat([S50H22, S50M22, S50U22, S50Z22], axis=1)
Options_2022

,S50H22,S50M22,S50U22,S50Z22
0,14.953988,13.968481,14.285714,15.548282
1,12.837838,13.829787,14.025245,14.913658
2,11.845387,14.749263,13.605442,14.660494
3,13.319672,14.705882,13.869626,14.913658
4,15.258216,13.693820,14.577259,13.888889
...,...,...,...,...
119,56.179775,0.493512,0.495788,78.629032
120,62.111801,NaN,NaN,125.000000
121,82.661290,NaN,NaN,185.185185
122,90.707965,NaN,NaN,0.496061


In [15]:
# 2023
Futures = S50H23
Futures_symbol = 'S50H23'
S50H23 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50M23
Futures_symbol = 'S50M23'
S50M23 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50U23
Futures_symbol = 'S50U23'
S50U23 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Futures = S50Z23
Futures_symbol = 'S50Z23'
S50Z23 = process_options_data(Futures_symbol)[['Normalized Premium']].rename(columns={'Normalized Premium': Futures_symbol})

Options_2023 = pd.concat([S50H23, S50M23, S50U23, S50Z23], axis=1)
Options_2023

,S50H23,S50M23,S50U23,S50Z23
0,18.026565,21.276596,11.309524,13.392857
1,16.964286,18.416206,13.504155,13.214286
2,15.289256,18.248175,12.549538,13.290230
3,16.047297,18.416206,14.503817,13.405797
4,14.752791,18.148820,13.286713,13.444767
...,...,...,...,...
119,52.197802,78.947368,110.119048,NaN
120,55.555556,121.621622,0.493454,NaN
121,67.708333,0.496963,NaN,NaN
122,108.333333,NaN,NaN,NaN


In [16]:
# Combined
Normalized_Premium = pd.concat([Options_2013, Options_2014, Options_2015,
                                Options_2016, Options_2017, Options_2018, Options_2019, 
                                Options_2020, Options_2021, Options_2022, Options_2023], 
                               axis=1)
Normalized_Premium

,S50U13,S50Z13,S50H14,S50M14,S50U14,S50Z14,S50H15,S50M15,S50U15,S50Z15,...,S50U21,S50Z21,S50H22,S50M22,S50U22,S50Z22,S50H23,S50M23,S50U23,S50Z23
0,11.668611,7.090909,6.709040,9.086189,13.309353,14.513788,19.195612,9.420290,14.338235,14.508929,...,11.271676,15.297907,14.953988,13.968481,14.285714,15.548282,18.026565,21.276596,11.309524,13.392857
1,11.822376,7.111597,6.732814,9.898190,11.131167,16.420361,21.694215,9.779338,13.017356,13.429752,...,12.148338,15.625000,12.837838,13.829787,14.025245,14.913658,16.964286,18.416206,13.504155,13.214286
2,11.877173,7.132407,6.629449,inf,11.038186,14.836795,22.483940,9.979529,14.409222,13.908702,...,12.500000,15.372168,11.845387,14.749263,13.605442,14.660494,15.289256,18.248175,12.549538,13.290230
3,11.904762,8.051197,6.780871,8.667389,11.533666,17.699115,18.388792,10.284810,14.025245,13.636364,...,13.087248,14.660494,13.319672,14.705882,13.869626,14.913658,16.047297,18.416206,14.503817,13.405797
4,11.560694,7.105460,6.794425,8.957655,11.664565,18.382353,21.560575,9.842520,14.880952,12.907609,...,13.031550,15.079365,15.258216,13.693820,14.577259,13.888889,14.752791,18.148820,13.286713,13.444767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,65.000000,NaN,75.203252,0.499156,0.498812,47.235023,59.880240,NaN,NaN,37.844037,...,NaN,130.000000,82.661290,NaN,NaN,185.185185,67.708333,0.496963,NaN,NaN
122,0.504418,NaN,71.705426,NaN,NaN,57.803468,80.000000,NaN,NaN,45.329670,...,NaN,103.723404,90.707965,NaN,NaN,0.496061,108.333333,NaN,NaN,NaN
123,NaN,NaN,0.495108,NaN,NaN,53.763441,0.500947,NaN,NaN,50.000000,...,NaN,0.493771,0.501026,NaN,NaN,NaN,0.503397,NaN,NaN,NaN
124,NaN,NaN,NaN,NaN,NaN,0.499860,NaN,NaN,NaN,61.111111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
Normalized_Premium.replace('inf', np.inf, inplace=True)
Normalized_Premium.fillna(method='bfill', inplace=True)
Normalized_Premium

,S50U13,S50Z13,S50H14,S50M14,S50U14,S50Z14,S50H15,S50M15,S50U15,S50Z15,...,S50U21,S50Z21,S50H22,S50M22,S50U22,S50Z22,S50H23,S50M23,S50U23,S50Z23
0,11.668611,7.090909,6.709040,9.086189,13.309353,14.513788,19.195612,9.420290,14.338235,14.508929,...,11.271676,15.297907,14.953988,13.968481,14.285714,15.548282,18.026565,21.276596,11.309524,13.392857
1,11.822376,7.111597,6.732814,9.898190,11.131167,16.420361,21.694215,9.779338,13.017356,13.429752,...,12.148338,15.625000,12.837838,13.829787,14.025245,14.913658,16.964286,18.416206,13.504155,13.214286
2,11.877173,7.132407,6.629449,inf,11.038186,14.836795,22.483940,9.979529,14.409222,13.908702,...,12.500000,15.372168,11.845387,14.749263,13.605442,14.660494,15.289256,18.248175,12.549538,13.290230
3,11.904762,8.051197,6.780871,8.667389,11.533666,17.699115,18.388792,10.284810,14.025245,13.636364,...,13.087248,14.660494,13.319672,14.705882,13.869626,14.913658,16.047297,18.416206,14.503817,13.405797
4,11.560694,7.105460,6.794425,8.957655,11.664565,18.382353,21.560575,9.842520,14.880952,12.907609,...,13.031550,15.079365,15.258216,13.693820,14.577259,13.888889,14.752791,18.148820,13.286713,13.444767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,65.000000,NaN,75.203252,0.499156,0.498812,47.235023,59.880240,NaN,NaN,37.844037,...,NaN,130.000000,82.661290,NaN,NaN,185.185185,67.708333,0.496963,NaN,NaN
122,0.504418,NaN,71.705426,NaN,NaN,57.803468,80.000000,NaN,NaN,45.329670,...,NaN,103.723404,90.707965,NaN,NaN,0.496061,108.333333,NaN,NaN,NaN
123,NaN,NaN,0.495108,NaN,NaN,53.763441,0.500947,NaN,NaN,50.000000,...,NaN,0.493771,0.501026,NaN,NaN,NaN,0.503397,NaN,NaN,NaN
124,NaN,NaN,NaN,NaN,NaN,0.499860,NaN,NaN,NaN,61.111111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
inf_values = np.isinf(Normalized_Premium)
if inf_values.any().any():
    Normalized_Premium.replace([np.inf, -np.inf], np.nan, inplace=True)
    Normalized_Premium.fillna(method='bfill', inplace=True)
else:
    pass

In [19]:
nan_counts = Normalized_Premium.isna().sum()

for column in Normalized_Premium.columns:
    if nan_counts[column] > 0:
        Normalized_Premium[column] = Normalized_Premium[column].shift(nan_counts[column])
        Normalized_Premium[column].fillna(0, inplace=True)

last_column = Normalized_Premium.columns[-1]
Normalized_Premium[last_column] = Normalized_Premium[last_column].shift(-14)
Normalized_Premium[last_column].fillna(0, inplace=True)
Normalized_Premium = Normalized_Premium.iloc[:-1, :]
Normalized_Premium

,S50U13,S50Z13,S50H14,S50M14,S50U14,S50Z14,S50H15,S50M15,S50U15,S50Z15,...,S50U21,S50Z21,S50H22,S50M22,S50U22,S50Z22,S50H23,S50M23,S50U23,S50Z23
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.508929,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,14.513788,0.000000,0.000000,0.000000,13.429752,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.392857
2,0.000000,0.000000,6.709040,0.000000,0.000000,16.420361,19.195612,0.000000,0.000000,13.908702,...,0.000000,15.297907,14.953988,0.000000,0.000000,0.000000,18.026565,0.000000,0.000000,13.214286
3,11.668611,0.000000,6.732814,0.000000,0.000000,14.836795,21.694215,0.000000,0.000000,13.636364,...,0.000000,15.625000,12.837838,0.000000,0.000000,15.548282,16.964286,0.000000,0.000000,13.290230
4,11.822376,0.000000,6.629449,9.086189,13.309353,17.699115,22.483940,0.000000,0.000000,12.907609,...,0.000000,15.372168,11.845387,0.000000,0.000000,14.913658,15.289256,21.276596,0.000000,13.405797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,33.064516,22.352941,54.093567,55.397727,65.950920,52.295918,58.823529,47.560976,45.000000,39.603960,...,55.084746,55.397727,61.349693,52.777778,67.114094,85.526316,51.351351,53.779070,70.075758,0.000000
121,36.931818,23.299748,52.631579,60.559006,61.764706,50.995025,55.865922,54.775281,45.685279,37.844037,...,59.451220,68.661972,56.179775,62.500000,87.837838,84.051724,52.197802,55.722892,75.203252,0.000000
122,35.454545,23.963731,51.388889,62.903226,68.181818,47.235023,67.114094,65.789474,50.000000,45.329670,...,72.222222,103.723404,62.111801,82.608696,92.857143,78.629032,55.555556,58.064516,101.648352,0.000000
123,48.994975,24.865591,75.203252,59.090909,79.629630,57.803468,59.880240,73.529412,49.180328,50.000000,...,99.489796,130.000000,82.661290,85.585586,110.795455,125.000000,67.708333,78.947368,97.368421,0.000000


In [20]:
sd_values = [-2.0, -1.5, -1.0, -0.5, 0, 0.5, 1.0, 1.5, 2.0]
sd_dict = {}

for sd_value in sd_values:
    label = f'{sd_value}SD'
    sd_dict[label] = Normalized_Premium[Normalized_Premium != 0].apply(lambda x: x.mean() + sd_value * x.std(), axis=1)

sd_dict['min'] = Normalized_Premium[Normalized_Premium != 0].min(axis=1)
sd_dict['max'] = Normalized_Premium[Normalized_Premium != 0].max(axis=1)
sd_dict['median'] = Normalized_Premium[Normalized_Premium != 0].median(axis=1)
sd_dict['mean'] = Normalized_Premium[Normalized_Premium != 0].mean(axis=1)

# Add the calculated values to the DataFrame
for label, values in sd_dict.items():
    Normalized_Premium[label] = values

Normalized_Premium = Normalized_Premium.iloc[:,-13:]
Normalized_Premium = Normalized_Premium[['min', '-2.0SD', '-1.5SD', '-1.0SD', '-0.5SD',	'median', 'mean', '0.5SD',	'1.0SD', '1.5SD', '2.0SD', 'max']]
Normalized_Premium = Normalized_Premium.rename(columns={'min':'Minimum', 
                                                        '-2.0SD':'-2.0 S.D.', '-1.5SD':'-1.5 S.D.', '-1.0SD':'-1.0 S.D.', '-0.5SD':'-0.5 S.D.', 
                                                        'median':'Median', 'mean':'Mean',
                                                        '0.5SD':'+0.5 S.D.', '1.0SD':'+1.0 S.D.', '1.5SD':'+1.5 S.D.', '2.0SD':'+2.0 S.D.',
                                                        'max':'Maximum'})

Remaining_day = pd.DataFrame({'Remaining Day': range(125, 0, -1)})
Normalized_Premium = pd.concat([Remaining_day, Normalized_Premium], axis=1)
Normalized_Premium

,Remaining Day,Minimum,-2.0 S.D.,-1.5 S.D.,-1.0 S.D.,-0.5 S.D.,Median,Mean,+0.5 S.D.,+1.0 S.D.,+1.5 S.D.,+2.0 S.D.,Maximum
0,125,9.626274,5.162485,6.888764,8.615043,10.341322,12.067601,12.067601,13.793880,15.520159,17.246438,18.972718,14.508929
1,124,9.423503,7.410430,8.869382,10.328334,11.787287,13.411305,13.246239,14.705192,16.164144,17.623097,19.082049,18.668831
2,123,6.709040,6.915341,8.724530,10.533718,12.342906,14.431345,14.152095,15.961283,17.770472,19.579660,21.388848,19.524618
3,122,6.732814,3.690771,6.765311,9.839851,12.914391,15.192538,15.988931,19.063471,22.138011,25.212551,28.287091,37.037037
4,121,6.629449,2.710315,5.950510,9.190706,12.430901,14.197531,15.671096,18.911291,22.151486,25.391681,28.631876,41.322314
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,5,12.640449,16.657869,26.540838,36.423807,46.306776,54.093567,56.189745,66.072715,75.955684,85.838653,95.721622,120.481928
121,4,13.861386,15.121139,26.377545,37.633951,48.890356,55.865922,60.146762,71.403168,82.659574,93.915979,105.172385,135.135135
122,3,19.078947,13.614302,26.833258,40.052215,53.271171,64.371257,66.490127,79.709084,92.928040,106.146996,119.365953,163.934426
123,2,21.067416,13.582434,29.663373,45.744312,61.825250,73.529412,77.906189,93.987128,110.068067,126.149006,142.229945,188.679245


In [21]:
Normalized_Premium.to_csv('Normalized_Premium.csv')